In [ ]:
# customer_service_server.py
from mcp_base_server import MCPBaseServer
import requests

class CustomerServiceServer(MCPBaseServer):
    def __init__(self):
        super().__init__("customer-service", "retail")
        self.crm_endpoint = "https://crm.example.com/api/v1"
        self.crm_token = self.get_secret("CRM_API_TOKEN")

    def get_expected_audience(self) -> str:
        return "api://customer-service"

    def validate_authorization(self, request_payload: dict):
        required_scopes = ["customer:read"]
        token_scopes = request_payload.get("scp", "").split()

        if not all(scope in token_scopes for scope in required_scopes):
            raise PermissionError("Missing required customer:read scope")

    def fetch_data(self, request_payload: dict):
        customer_id = request_payload.get("customer_id")
        if not customer_id:
            raise ValueError("Missing customer_id in request")

        response = requests.get(
            f"{self.crm_endpoint}/customers/{customer_id}",
            headers={"Authorization": f"Bearer {self.crm_token}"},
            timeout=5
        )
        response.raise_for_status()
        return response.json()

    def build_context(self, raw_data) -> dict:
        return {
            "@context": "https://schema.org",
            "@type": "Person",
            "identifier": raw_data["id"],
            "name": raw_data["name"],
            "email": raw_data["email"],
            "telephone": raw_data["phone"],
            "address": {
                "@type": "PostalAddress",
                "streetAddress": raw_data["address"]["street"],
                "addressLocality": raw_data["address"]["city"],
                "postalCode": raw_data["address"]["zip"],
                "addressCountry": raw_data["address"]["country"]
            }
        }